# Adversarial Attacks Evaluation

Evaluate robustness of baseline and lottery tickets under adversarial attacks.

In [1]:
import sys
sys.path.append('..')

import numpy as np
import matplotlib.pyplot as plt
import torch
import json

from src.utils.config import load_config, get_device
from src.models.deep_hedging import DeepHedgingNetwork
from src.data.preprocessor import create_dataloaders
from src.evaluation.metrics import evaluate_robustness
from src.utils.visualization import plot_robustness_comparison

## Setup

In [2]:
config = load_config('../config.yaml')
device = get_device(config)

# Load test data
S_test = np.load('../data/processed/S_test.npy')
v_test = np.load('../data/processed/v_test.npy')
Z_test = np.load('../data/processed/Z_test.npy')

# Dummy train/val for dataloader
S_train = np.load('../data/processed/S_train.npy')[:1000]
v_train = np.load('../data/processed/v_train.npy')[:1000]
Z_train = np.load('../data/processed/Z_train.npy')[:1000]

batch_size = config['training']['batch_size'] or 256
_, _, test_loader = create_dataloaders(
    S_train, v_train, Z_train, S_train, v_train, Z_train, S_test, v_test, Z_test,
    batch_size, config['compute']['num_parallel_workers']
)

K = config['data']['heston']['K']
T = config['data']['T']
dt = config['data']['dt']

🔍 Auto-detected device: cpu
 Using device: cpu


## Load Models

In [ ]:
models_to_test = {}

# Baseline dense
model_dense = DeepHedgingNetwork(config['model'])
model_dense.load_state_dict(torch.load('../experiments/baseline/best_model.pt', map_location=device))
model_dense = model_dense.to(device)
models_to_test['dense_baseline'] = model_dense

# Sparse tickets (if available)
for sparsity in [0.5, 0.6, 0.7, 0.8, 0.9]:
    model_path = f'../experiments/pruning/sparsity_{int(sparsity*100)}/model.pt'
    try:
        model_sparse = DeepHedgingNetwork(config['model'])
        model_sparse.load_state_dict(torch.load(model_path, map_location=device))
        model_sparse = model_sparse.to(device)
        models_to_test[f'ticket_{int(sparsity*100)}'] = model_sparse
    except:
        print(f"Model for sparsity {sparsity} not found, skipping")

print(f"Loaded {len(models_to_test)} models")

## Evaluate Robustness

In [ ]:
results_attacks = {}

for model_name, model in models_to_test.items():
    print(f"\nEvaluating {model_name}...")
    
    results = evaluate_robustness(model, test_loader, config, K, T, dt, device)
    results_attacks[model_name] = results
    
    print(f"  Clean CVaR: {results['clean']['cvar_005']:.6f}")
    print(f"  FGSM CVaR: {results['fgsm']['cvar_005']:.6f}")
    print(f"  PGD-10 CVaR: {results['pgd10']['cvar_005']:.6f}")
    print(f"  Robustness Gap (PGD-10): {results['robustness_gap_pgd10']:.6f}")

# Save results
with open('../experiments/adversarial/attack_results.json', 'w') as f:
    json.dump(results_attacks, f, indent=2)

print("\nResults saved")

## Visualization

In [ ]:
model_names = list(results_attacks.keys())
clean_cvars = [results_attacks[m]['clean']['cvar_005'] for m in model_names]
fgsm_cvars = [results_attacks[m]['fgsm']['cvar_005'] for m in model_names]
pgd_cvars = [results_attacks[m]['pgd10']['cvar_005'] for m in model_names]

plot_robustness_comparison(
    model_names,
    clean_cvars,
    fgsm_cvars,
    pgd_cvars,
    title="Robustness Comparison: Clean vs FGSM vs PGD",
    save_path='../figures/robustness_comparison.pdf'
)

## Summary

Standard lottery tickets are vulnerable to adversarial attacks. Robustness gap increases with sparsity.